In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

import warnings
warnings.simplefilter("ignore")

import random

random.seed(23)

In [2]:
women = pd.read_csv("data/sa_women.2.clean.csv")
women.head()

,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_02,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,lat,lon,NL
0,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,0.005750,...,0.019968,0.002848,0.007537,0.000000,0.012928,0,0,-29.682270,24.734743,0.292039
1,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,0.007575,...,0.018946,0.014566,0.057127,0.019092,0.004131,0,0,-29.119311,24.757737,3.207775
2,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,0.006686,...,0.083010,0.057560,0.010358,0.001421,0.040881,0,0,-29.142276,25.094093,0.000000
3,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,0.005255,...,0.002689,0.000000,0.000669,0.000000,0.005011,0,0,-29.372052,24.942867,2.038778
4,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,0.004985,...,0.009699,0.004859,0.001290,0.000673,0.017629,0,0,-29.409381,25.290165,0.000000


In [3]:
women.columns

Index(['total_households', 'total_individuals', 'target', 'dw_00', 'dw_01',
       'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09',
       'dw_10', 'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01', 'psa_02',
       'psa_03', 'psa_04', 'stv_00', 'car_00', 'lln_00', 'lan_00', 'lan_01',
       'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06', 'lan_07', 'lan_08',
       'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_13', 'lan_14', 'pg_00',
       'pg_01', 'pg_02', 'pg_03', 'pg_04', 'lgt_00', 'pw_00', 'pw_01', 'pw_02',
       'pw_03', 'pw_04', 'pw_05', 'pw_06', 'pw_07', 'pw_08', 'lat', 'lon',
       'NL'],
      dtype='object')

In [3]:
## Let's begin by making a validation set to test the multiple types of models on

from sklearn.model_selection import train_test_split

X = women.drop("target", axis=1)
y = women["target"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
## At the start we're going to run a basic linear regression on the raw X_train data to get a baseline idea about
## model performance

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

print("Linear regression on the base data set:")

linreg = LinearRegression(fit_intercept=False)
print(cross_val_score(estimator=linreg, X=X_train, y=y_train, scoring="r2", cv=5).mean())
print(np.sqrt(-cross_val_score(estimator=linreg, X=X_train, y=y_train, scoring="neg_mean_squared_error").mean()))

Linear regression on the base data set:
0.8576268108992714
3.909750875419991


In [10]:
## We see that a basic linear regression already outputs a good prediction, explaining a lot of the model's variance.
## Let's try scaling the input features and using different models to see if we can improve on it

from sklearn.preprocessing import StandardScaler, RobustScaler, power_transform

stan_scaler = StandardScaler()
rob_scaler = RobustScaler()

X_train_stanscale = stan_scaler.fit_transform(X_train)
X_train_robscale = rob_scaler.fit_transform(X_train)
X_train_powscale = power_transform(X=X_train, method='yeo-johnson')

print("Scaled data with regular linear regression:")

print("Standard Scaled R^2:", cross_val_score(estimator=linreg, X=X_train_stanscale, y=y_train, scoring="r2", cv=5).mean())
print("Robust Scaled R^2:", cross_val_score(estimator=linreg, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())
print("Power Scaled R^2:", cross_val_score(estimator=linreg, X=X_train_powscale, y=y_train, scoring="r2", cv=5).mean())

Scaled data with regular linear regression:
Standard Scaled R^2: -5.089423412521885
Robust Scaled R^2: 0.8571139104845855
Power Scaled R^2: -5.053218528586572


In [7]:
## Let's see how a regularized linear model does on the different standardized datasets
from sklearn.linear_model import ElasticNet

net = ElasticNet()

print("Standard Scaled R^2:", cross_val_score(estimator=net, X=X_train_stanscale, y=y_train, scoring="r2", cv=5).mean())
print("Robust Scaled R^2:", cross_val_score(estimator=net, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())
print("Power Scaled R^2:", cross_val_score(estimator=net, X=X_train_powscale, y=y_train, scoring="r2", cv=5).mean())

Standard Scaled R^2: 0.7932164348774753
Robust Scaled R^2: 0.7866920136128908
Power Scaled R^2: 0.8096108087204051


In [59]:
## Let's see if polynomial features increases model performance

from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2, include_bias=False)

X_train_poly = poly.fit_transform(X_train)
X_train_polystan = poly.fit_transform(X_train_stanscale)
X_train_polyrob = poly.fit_transform(X_train_robscale)
X_train_polypow = poly.fit_transform(X_train_powscale)

## First let's try the polynomial features with the elasticnet regressor

print("With polynomial features, elastic net:")

print("Unscaled R^2:", cross_val_score(estimator=net, X=X_train_poly, y=y_train, scoring="r2", cv=5).mean())
print("Standard Scaled R^2:", cross_val_score(estimator=net, X=X_train_polystan, y=y_train, scoring="r2", cv=5).mean())
print("Robust Scaled R^2:", cross_val_score(estimator=net, X=X_train_polyrob, y=y_train, scoring="r2", cv=5).mean())
print("Power Scaled R^2:", cross_val_score(estimator=net, X=X_train_polypow, y=y_train, scoring="r2", cv=5).mean())

With polynomial features, elastic net:
Unscaled R^2: 0.8458208727022052
Standard Scaled R^2: 0.8178356698038547
Robust Scaled R^2: 0.8125564614236602
Power Scaled R^2: 0.8354024445294448


In [9]:
## Now with the regular linear regression
print("Polynomial features with regular linear regression")
print("Unscaled R^2:", cross_val_score(estimator=linreg, X=X_train_poly, y=y_train, scoring="r2", cv=5).mean())
print("Standard Scaled R^2:", cross_val_score(estimator=linreg, X=X_train_polystan, y=y_train, scoring="r2", cv=5).mean())
print("Robust Scaled R^2:", cross_val_score(estimator=linreg, X=X_train_polyrob, y=y_train, scoring="r2", cv=5).mean())
print("Power Scaled R^2:", cross_val_score(estimator=linreg, X=X_train_polypow, y=y_train, scoring="r2", cv=5).mean())

Polynomial features with regular linear regression
Unscaled R^2: -2.985200643007221
Standard Scaled R^2: -65.35270332952197
Robust Scaled R^2: -54.777408061260225
Power Scaled R^2: 0.32446339357296083


In [6]:
## It seems our best performance with a linear regressor was with the unscaled X_train with vanilla linear regression.
## Let's check and save its performance on the validation set

from sklearn.metrics import r2_score, mean_squared_error

valid_r2 = {}
valid_rmse = {}

linreg.fit(X_train, y_train)
predictions = linreg.predict(X_valid)

valid_r2["unscaled_linreg"] = r2_score(y_valid, predictions)
valid_rmse["unscaled_linreg"] = np.sqrt(mean_squared_error(y_valid, predictions))

print("Linear regression's performance on the validation set:")
print(valid_r2)
print(valid_rmse)

Linear regression's performance on the validation set:
{'unscaled_linreg': 0.8592054386963557}
{'unscaled_linreg': 3.726941696442016}


In [5]:
## Now let's see how a random forest performs on this data, and this can also give us some idea of feature importances

from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()

print("Random forest:")
print(cross_val_score(estimator=forest, X=X_train, y=y_train, scoring="r2", cv=5).mean())
print(np.sqrt(-cross_val_score(estimator=forest, X=X_train, y=y_train, scoring="neg_mean_squared_error", cv=5).mean()))

Random forest:
0.8774508434485696
3.6422804609833395


In [12]:
## Wow, this is very promising! Out of the box, the random forest does better on cross validation than linear regression
## Now, let's see the random forest model's feature importances

forest.fit(X_train, y_train)
forest.feature_importances_

array([0.00218614, 0.00255499, 0.00267956, 0.00432479, 0.00281835,
       0.00174957, 0.00158875, 0.00179831, 0.00247225, 0.00269799,
       0.00275604, 0.00195806, 0.00146929, 0.00309955, 0.        ,
       0.        , 0.57607372, 0.00639669, 0.00196505, 0.00217788,
       0.005652  , 0.00432249, 0.20044678, 0.01199845, 0.00324895,
       0.00315159, 0.0021525 , 0.00510636, 0.00357126, 0.00366679,
       0.00424451, 0.00433272, 0.00265586, 0.00400614, 0.00311076,
       0.00497131, 0.00273008, 0.        , 0.00133011, 0.00872881,
       0.00506755, 0.00279782, 0.00523709, 0.0030801 , 0.00295414,
       0.04679625, 0.00310661, 0.00337593, 0.00234871, 0.00249449,
       0.00185137, 0.0023738 , 0.        , 0.        , 0.00497006,
       0.00869577, 0.00465585])

In [13]:
importances = {feature: importance for feature, importance in zip(X_train.columns, forest.feature_importances_)}
feature_importances = pd.DataFrame.from_dict(importances, orient='index', columns=["feature_importance"])
feature_importances.sort_values("feature_importance", ascending=False).head()

,feature_importance
psa_00,0.576074
car_00,0.200447
pw_00,0.046796
lln_00,0.011998
pg_00,0.008729


In [14]:
## Now let's try using grid search to optimize the random forest

from sklearn.model_selection import GridSearchCV

forestgrid = {
    "n_estimators": [10, 50, 100, 1000, 10000],
    "max_depth": [3, 4, 5, 6, None],
    "max_features": ["auto", "sqrt", "log2"],
    "bootstrap": [True, False],
}

forest = RandomForestRegressor()

grid = GridSearchCV(estimator=forest, param_grid=forestgrid, scoring="r2", n_jobs=-1, verbose=2, cv=3)

In [15]:
%%time
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed: 30.5min finished


Wall time: 32min 27s


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [3, 4, 5, 6, None],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 50, 100, 1000, 10000]},
             scoring='r2', verbose=2)

In [16]:
grid.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'n_estimators': 10000}

In [17]:
grid.best_score_

0.8765206234057411

In [8]:
## Now that we have optimized hyperparameters for the random forest, let's evaluate its importance.

best_forest = RandomForestRegressor(bootstrap=False, max_depth=None, max_features='sqrt', n_estimators=10000)

best_forest.fit(X_train, y_train)
predictions = best_forest.predict(X_valid)

valid_r2["optimized_forest"] = r2_score(y_valid, predictions)
valid_rmse["optimized_forest"] = np.sqrt(mean_squared_error(y_valid, predictions))

print(valid_r2)
print(valid_rmse)

{'unscaled_linreg': 0.8592054386963557, 'optimized_forest': 0.8901687135107363}
{'unscaled_linreg': 3.726941696442016, 'optimized_forest': 3.291718548192841}


In [19]:
##Now, let's explore other potential regression model candidates, and find the most promising ones using cv score

from sklearn import svm

svreg = svm.SVR(kernel='linear')

print(cross_val_score(estimator=svreg, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())
print(np.sqrt(-cross_val_score(estimator=svreg, X=X_train_robscale, y=y_train, scoring="neg_mean_squared_error", cv=5).mean()))

0.8574056842129044
3.914679087104853


In [11]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(n_estimators=1000)

print(cross_val_score(estimator=gb, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())

0.8887914495174044


In [16]:
## The gradient boosting regressor seems incredibly alluring with that cross-val score! So let's optimize the 
## hyperparameters using RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

gb_grid = {
    "n_estimators": [100, 1000, 10000],
    "learning_rate": loguniform(0.001, 1),
    "max_features": ["auto", "sqrt", "log2"],
    "max_depth": [2, 3, 4, 5],
}

gb = GradientBoostingRegressor()
rand = RandomizedSearchCV(estimator=gb, param_distributions=gb_grid, scoring="r2", n_jobs=-1, 
                          verbose=2, cv=3, n_iter=100)

rand.fit(X_train_robscale, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 20.2min finished


RandomizedSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000286EA4B5148>,
                                        'max_depth': [2, 3, 4, 5],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': [100, 1000, 10000]},
                   scoring='r2', verbose=2)

In [17]:
rand.best_params_

{'learning_rate': 0.005557832351846306,
 'max_depth': 5,
 'max_features': 'sqrt',
 'n_estimators': 10000}

In [19]:
rand.best_score_

0.8922381016522563

In [28]:
## Now let's evaluate the optimized gradient boosting regressor on the validation set
## Since we used the robustly scaled data to find the optimized hyperparameters, we will use robust scaling on the
## validation set to evaluate the model's performance

X_valid_robscale = rob_scaler.transform(X_valid)

best_gb = GradientBoostingRegressor(learning_rate=0.0056, max_depth=5, max_features='sqrt', n_estimators=10000)

best_gb.fit(X_train_robscale, y_train)
predictions = best_gb.predict(X_valid_robscale)

valid_r2["optimized_gradboost"] = r2_score(y_valid, predictions)
valid_rmse["optimized_gradboost"] = np.sqrt(mean_squared_error(y_valid, predictions))

print(valid_r2)
print(valid_rmse)

{'unscaled_linreg': 0.8592054386963557, 'optimized_forest': 0.8901687135107363, 'optimized_gradboost': 0.9046246723318132, 'optimized_baggingextratrees': 0.8950191309619571}
{'unscaled_linreg': 3.726941696442016, 'optimized_forest': 3.291718548192841, 'optimized_gradboost': 3.067451331003651, 'optimized_baggingextratrees': 3.218212675846616}


In [31]:
## We see the gradient boosting model has even better performance on the validation set! Before we wrap up, let's see
## if there are any other distinctly promising models

from sklearn.linear_model import PoissonRegressor

psr = PoissonRegressor(alpha=0.5)

print(cross_val_score(estimator=psr, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())

0.85538074694879


In [55]:
from sklearn.linear_model import Ridge

rid = Ridge()

print(cross_val_score(estimator=rid, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())

0.8576030284557467


In [57]:
from sklearn.linear_model import BayesianRidge

bayrid = BayesianRidge()

print(cross_val_score(estimator=bayrid, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())

0.8576069179706138


In [21]:
from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.tree import ExtraTreeRegressor

estimator_bagging_random_tree_1000 = BaggingRegressor(n_estimators=1000,
                                    base_estimator=ExtraTreeRegressor())
print(cross_val_score(estimator=estimator_bagging_random_tree_1000, X=X_train_robscale, y=y_train, scoring="r2", cv=5).mean())

0.883311250520606


In [22]:
## The Bagging Regressor, using the Extra Trees Regressor as the base estimator, gives us 

bagtreeparams = {
    "bootstrap": [True, False],
    "bootstrap_features": [True, False],
    "n_estimators": [10, 50, 100, 1000, 10000]
}

bagtree_reg = BaggingRegressor(n_estimators=1000, base_estimator=ExtraTreeRegressor())

bagtree_grid = GridSearchCV(estimator=bagtree_reg, param_grid=bagtreeparams, scoring="r2", n_jobs=-1, verbose=2, cv=3)
bagtree_grid.fit(X_train_robscale, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  5.4min finished


GridSearchCV(cv=3,
             estimator=BaggingRegressor(base_estimator=ExtraTreeRegressor(),
                                        n_estimators=1000),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'n_estimators': [10, 50, 100, 1000, 10000]},
             scoring='r2', verbose=2)

In [23]:
bagtree_grid.best_params_

{'bootstrap': False, 'bootstrap_features': False, 'n_estimators': 10000}

In [24]:
bagtree_grid.best_score_

0.8848817579887772

In [29]:
best_bagtree = BaggingRegressor(n_estimators=10000, bootstrap=False, base_estimator=ExtraTreeRegressor())

best_bagtree.fit(X_train_robscale, y_train)
predictions = best_bagtree.predict(X_valid_robscale)

valid_r2["optimized_baggingextratrees"] = r2_score(y_valid, predictions)
valid_rmse["optimized_baggingextratrees"] = np.sqrt(mean_squared_error(y_valid, predictions))

print(valid_r2)
print(valid_rmse)

{'unscaled_linreg': 0.8592054386963557, 'optimized_forest': 0.8901687135107363, 'optimized_gradboost': 0.9046246723318132, 'optimized_baggingextratrees': 0.8954026174887673}
{'unscaled_linreg': 3.726941696442016, 'optimized_forest': 3.291718548192841, 'optimized_gradboost': 3.067451331003651, 'optimized_baggingextratrees': 3.212329364294906}


In [30]:
## So we see that the gradientboostingregressor offers us the best performance on the validation set, and so we will 
## use this to generate predictions on the test set

rob_scaler = RobustScaler()

Xtrain_robscale = rob_scaler.fit_transform(X)

best_gb = GradientBoostingRegressor(learning_rate=0.0056, max_depth=5, max_features='sqrt', n_estimators=10000)

best_gb.fit(Xtrain_robscale, y)

GradientBoostingRegressor(learning_rate=0.0056, max_depth=5,
                          max_features='sqrt', n_estimators=10000)

In [31]:
## Let's bring in the test set

test_set = pd.read_csv("data/test.csv")
test_set.head()

,ward,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,21001001: Ward 1,2504.95194,8745.15151,0.947257,0.000873,0.002021,0.000000,0.000000,0.030116,0.000452,...,0.001757,0.000000,0.0,0.006649,0.0,0.0,ZA2101001,-32.637758,23.848688,0.000000
1,21001002: Ward 2,2080.27718,7258.11764,0.844993,0.000481,0.043629,0.004714,0.012323,0.012300,0.022132,...,0.000691,0.000000,0.0,0.002916,0.0,0.0,ZA2101002,-31.990536,24.555818,0.000000
2,21001003: Ward 3,1106.62639,5919.13170,0.651380,0.007937,0.007113,0.000000,0.001977,0.259711,0.006505,...,0.002253,0.000000,0.0,0.000000,0.0,0.0,ZA2101003,-32.283595,24.563940,8.269556
3,21001004: Ward 4,2175.56096,10280.57452,0.410837,0.002468,0.011511,0.000485,0.000000,0.449604,0.009256,...,0.000000,0.000661,0.0,0.001379,0.0,0.0,ZA2101004,-32.261612,24.542202,8.626625
4,21001005: Ward 5,1270.83883,6018.34202,0.942851,0.002638,0.000821,0.000000,0.000891,0.000787,0.000830,...,0.000000,0.000000,0.0,0.001660,0.0,0.0,ZA2101005,-32.251571,24.558537,8.601754


In [32]:
## Now let's transform and scale the test set so that the optimized gradient boosting model will work on it
test_set.columns

Index(['ward', 'total_households', 'total_individuals', 'dw_00', 'dw_01',
       'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09',
       'dw_10', 'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01', 'psa_02',
       'psa_03', 'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01', 'lln_00',
       'lln_01', 'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05',
       'lan_06', 'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12',
       'lan_13', 'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04',
       'lgt_00', 'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05', 'pw_06',
       'pw_07', 'pw_08', 'ADM4_PCODE', 'lat', 'lon', 'NL'],
      dtype='object')

In [ ]:
test_set.set_index("ward")
test_set = test_set.drop(["stv_01", "car_01", "lln_01", "ADM4_PCODE"], axis=1)

In [35]:
test_set.shape

(1013, 58)

In [49]:
Xtest = test_set.reset_index()
Xtest.drop(["index", "ward"], axis=1, inplace=True)
Xtest_robscale = rob_scaler.transform(Xtest)

## Let's make our predictions 

predictions = best_gb.predict(Xtest_robscale)

test_set["target"] = predictions

test_set.head()

,ward,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,lat,lon,NL,target
0,21001001: Ward 1,2504.95194,8745.15151,0.947257,0.000873,0.002021,0.000000,0.000000,0.030116,0.000452,...,0.001757,0.000000,0.0,0.006649,0.0,0.0,-32.637758,23.848688,0.000000,17.570097
1,21001002: Ward 2,2080.27718,7258.11764,0.844993,0.000481,0.043629,0.004714,0.012323,0.012300,0.022132,...,0.000691,0.000000,0.0,0.002916,0.0,0.0,-31.990536,24.555818,0.000000,13.068645
2,21001003: Ward 3,1106.62639,5919.13170,0.651380,0.007937,0.007113,0.000000,0.001977,0.259711,0.006505,...,0.002253,0.000000,0.0,0.000000,0.0,0.0,-32.283595,24.563940,8.269556,21.137121
3,21001004: Ward 4,2175.56096,10280.57452,0.410837,0.002468,0.011511,0.000485,0.000000,0.449604,0.009256,...,0.000000,0.000661,0.0,0.001379,0.0,0.0,-32.261612,24.542202,8.626625,16.487480
4,21001005: Ward 5,1270.83883,6018.34202,0.942851,0.002638,0.000821,0.000000,0.000891,0.000787,0.000830,...,0.000000,0.000000,0.0,0.001660,0.0,0.0,-32.251571,24.558537,8.601754,20.935502


In [53]:
## And now output into the format that Zindi expects
test_set[["ward", "target"]].set_index("ward").to_csv("data/davshimsubmission1.csv")